In [584]:
#####Bayesian sparse stochastic block model#####
import numpy as np
import pandas as pd
import matplotlib.pyplot  as plt
import numpy.matlib
import gensim
import itertools
import scipy
from numpy.random import *
from scipy import optimize
from scipy import sparse
import seaborn as sns

In [585]:
##多項分布の乱数を生成する関数
def rmnom(pr, n, k, no):
    z_id = np.argmax((np.cumsum(pr, axis=1) > np.random.rand(n).reshape(n, 1)), axis=1)
    Z = sparse.coo_matrix((np.repeat(1, n), (no, np.array(z_id))), shape=(n, k))   #スパース行列の設定
    return Z

In [586]:
####データの発生####
##データの設定
N = 10000   #ユーザー数
K = 3000   #アイテム数
seg_u = 8   #ユーザーのセグメント数
seg_i = 7   #アイテムのセグメント数

In [587]:
##パラメータとセグメントを生成
#ユーザーセグメントを生成
alpha01 = np.repeat(3.0, seg_u)
pi1 = np.random.dirichlet(alpha01, 1).reshape(-1)
Z1 = np.random.multinomial(1, pi1, N)
z1_vec = np.dot(Z1, np.arange(seg_u))
z1_count = np.sum(Z1, axis=0)
mix1 = np.mean(Z1, axis=0)   #混合率の真値
pit1 = pi1
print(z1_count)
print(mix1)

[1401 1224 1269 2272  489  520  831 1994]
[0.1401 0.1224 0.1269 0.2272 0.0489 0.052  0.0831 0.1994]


In [588]:
#アイテムセグメントを生成
alpha02 = np.repeat(3.0, seg_i)
pi2 = np.random.dirichlet(alpha02, 1).reshape(-1)
Z2 = np.random.multinomial(1, pi2, K)
z2_vec = np.dot(Z2, np.arange(seg_i))
z2_count = np.sum(Z2, axis=0)
mix2 = np.mean(Z2, axis=0)   #混合率の真値
pit2 = pi2
print(z2_count)
print(mix2)

[394 872 416 249 305 384 380]
[0.13133333 0.29066667 0.13866667 0.083      0.10166667 0.128
 0.12666667]


In [589]:
##観測変数のパラメータの設定
#ベータ分布からパラメータ行列を生成
beta01 = 0.2
beta02 = 2.25
gamma = np.random.beta(beta01, beta02, seg_u*seg_i).reshape(seg_u, seg_i)
gammat = gamma

In [590]:
##ベルヌーイ分布から観測行列を生成
#ユーザー×アイテムの共起行列を生成

for i in range(seg_u):
    print(i)
    for j in range(seg_i):
        n = z1_count[i] * z2_count[j]
        Data[np.ix_(z1_vec==i, z2_vec==j)] = np.random.binomial(1, gamma[i, j], n).reshape(z1_count[i], z2_count[j])

0
1
2
3
4
5
6
7


In [591]:
#データを変換
Data = np.array(Data, dtype="int16")
Data_T = Data.T
sparse_data = sparse.csr_matrix(Data)
sparse_data_T = sparse_data.T

In [592]:
##多項分布のパラメータを設定
#ユーザーのパラメータを設定
theta = np.zeros((seg_u, seg_i))
for i in range(seg_u):
    n = np.sum(Data[z1_vec==i, ])
    for j in range(seg_i):
        theta[i, j] = np.sum(Data[z1_vec==i, ][:, z2_vec==j]) / n
thetat = theta

#アイテムのパラメータを設定
phi = np.zeros((seg_i, seg_u))
for i in range(seg_i):
    n = np.sum(Data[:, z2_vec==i])
    for j in range(seg_u):
        phi[i, j] = np.sum(Data[z1_vec==j, ][:, z2_vec==i]) / n
phit = phi

In [593]:
####マルコフ連鎖モンテカルロ法でBayesian sparse stochastic block modelを推定####
##アルゴリズムの設定
R = 5000
keep = 2
burnin = int(500/keep)
disp = 25
iter = 0
sbeta = 1.5

In [594]:
##事前分布の設定
tau = 1
alpha1 = 1
alpha2 = 1

In [595]:
##パラメータの真値
Zi1 = Z1
Zi2 = Z2
sparse_z1 = sparse.csr_matrix(Zi1)
sparse_z2 = sparse.csr_matrix(Zi2)
r1 = pit1
r2 = pit2
oldtheta = thetat
oldphi = phit
oldtheta = (oldtheta + 0.000001) / np.sum(oldtheta + 0.000001, axis=1).reshape(seg_u, 1)
oldphi = (oldphi + 0.000001) / np.sum(oldphi + 0.000001, axis=1).reshape(seg_i, 1)

In [596]:
##初期値の設定
#混合率の初期値
r1 = np.repeat(1/seg_u, seg_u)
r2 = np.repeat(1/seg_i, seg_i)

In [597]:
#ブロックごとのパラメータの初期値
index_u = np.append(np.arange(0, N, int(N/seg_u)), N-1)[0:(seg_u+1)]
index_i = np.append(np.arange(0, K, int(K/seg_i)), K-1)[0:(seg_i+1)]
sortlist1 = np.argsort(np.sum(Data, axis=1))
sortlist2 = np.argsort(np.sum(Data, axis=0))[::-1]

In [598]:
#クラス割当の初期値を設定
z1 = np.zeros(N, dtype="int")
z2 = np.zeros(K, dtype="int")

for i in range(index_u.shape[0]-1):
    #ユーザーのクラス割当のインデックスを設定
    index1 = sortlist1[index_u[i]:index_u[i+1]] 
    z1[index1] = i
    
    for j in range(index_i.shape[0]-1):
        #アイテムのクラス割当のインデックスを設定
        index2 = sortlist2[index_i[j]:index_i[j+1]]
        z2[index2] = j

In [599]:
#セグメントのインデックスを作成
index_n = np.array(range(N))
index_k = np.array(range(K))
index1 = [i for i in range(seg_u)]
index2 = [j for j in range(seg_i)]
Zi1 = np.zeros((N, seg_u), dtype="int")
Zi2 = np.zeros((K, seg_i), dtype="int")
n1 = np.zeros(seg_u, dtype="int")
n2 = np.zeros(seg_i, dtype="int")

for i in range(seg_u):
    index1[i] = index_n[z1==i]
    Zi1[index1[i], i] = 1
    n1[i] = np.sum(Data[index1[i], ])
for j in range(seg_i):
    index2[j] = index_k[z2==j]
    Zi2[index2[j], j] = 1
    n2[j] = np.sum(Data[:, index2[j]])

sparse_z1 = sparse.csr_matrix(Zi1)
sparse_z2 = sparse.csr_matrix(Zi2)

In [600]:
#ユーザーセグメントの初期パラメータ
oldtheta = np.zeros((seg_u, seg_i))
for i in range(seg_u):
    for j in range(seg_i-1):
        freq = np.sum(Data[np.ix_(index1[i], index2[j])])
        oldtheta[i, j] = freq / n1[i]
oldtheta[:, seg_i-1] = 1-np.sum(oldtheta, axis=1)
oldtheta = (oldtheta + 0.000001) / np.sum(oldtheta + 0.000001, axis=1).reshape(seg_u, 1)

#アイテムセグメントの初期パラメータ
oldphi = np.zeros((seg_i, seg_u))
for i in range(seg_i):
    for j in range(seg_u-1):
        freq = np.sum(Data[np.ix_(index1[j], index2[i])])
        oldphi[i, j] = freq / n2[i]
oldphi[:, seg_u-1] = 1-np.sum(oldphi, axis=1)
oldphi = (oldphi + 0.000001) / np.sum(oldphi + 0.000001, axis=1).reshape(seg_i, 1)

In [601]:
#ユーザー、アイテムごとの購買数
n_user = np.sum(Data, axis=1)
n_item = np.sum(Data, axis=1)

In [602]:
##パラメータの格納用配列
THETA = np.zeros((seg_u, seg_i, int(R/keep)))
PHI = np.zeros((seg_i, seg_u, int(R/keep)))
SEG1 = np.zeros((N, seg_u), dtype="int")
SEG2 = np.zeros((K, seg_i), dtype="int")

In [603]:
####ギブスサンプリングでパラメータをサンプリング####
for rp in range(R):
    
    ##ユーザーセグメント割当を生成
    #セグメントごとの多項分布の対数尤度を設定
    y1 = np.dot(sparse_data, sparse_z2).todense()
    LLi1 = np.array(np.dot(y1, np.log(oldtheta.T)))

    #セグメント割当確率を設定
    r_matrix = np.repeat(r1, N).reshape(N, seg_u, order="F")
    expl = r_matrix * np.exp(LLi1 - np.max(LLi1, axis=1).reshape(N, 1))
    z1_rate = expl / np.sum(expl, axis=1).reshape(N, 1)
    z1_rate = (z1_rate + 0.000001) / np.sum(z1_rate + 0.000001, axis=1).reshape(N, 1)
    
    #多項分布よりセグメントをサンプリング
    sparse_z1 = rmnom(z1_rate, N, seg_u, np.array(range(N))).tocsr()
    Zi1 = np.array(sparse_z1.todense())
    z1_vec = np.dot(Zi1, np.arange(seg_u))

    #混合率を更新
    z_sums = np.sum(Zi1, axis=0) + tau
    r1 = np.random.dirichlet(z_sums, 1).reshape(-1) 

    
    ##アイテムセグメント割当を生成
    y2 = np.dot(sparse_data_T, sparse_z1).todense()
    LLi2 = np.array(np.dot(y2, np.log(oldphi.T)))

    #セグメント割当確率を設定
    r_matrix = np.repeat(r2, K).reshape(K, seg_i, order="F")
    expl = r_matrix * np.exp(LLi2 - np.max(LLi2, axis=1).reshape(K, 1))
    z2_rate = expl / np.sum(expl, axis=1).reshape(K, 1)
    z2_rate = (z2_rate + 0.000001) / np.sum(z2_rate + 0.000001, axis=1).reshape(K, 1)
    
    #多項分布よりセグメントをサンプリング
    sparse_z2 = rmnom(z2_rate, K, seg_i, np.array(range(K))).tocsr()
    Zi2 = np.array(sparse_z2.todense())
    z2_vec = np.dot(Zi2, np.arange(seg_i))

    #混合率を更新
    z_sums = np.sum(Zi2, axis=0) + tau
    r2 = np.random.dirichlet(z_sums, 1).reshape(-1)


    ##ユーザーとアイテムのパラメータをサンプリング
    #セグメントインデックスを作成
    for i in range(seg_u):
        index1[i] = index_n[z1_vec==i]
    for j in range(seg_i):
        index2[j] = index_k[z2_vec==j]

    #ディリクリ分布のパラメータ
    freq_user = np.zeros((seg_u, seg_i))
    for i in range(seg_u):
        x = Data[index1[i], ]
        for j in range(seg_i):
            freq_user[i, j] = np.sum(x[:, index2[j]])
    freq_item = freq_user.T   #アイテムのパラメータはユーザーのパラメータを転置させるだけ

    #ディリクレ分布からパラメータをサンプリング
    oldtheta = np.array([np.random.dirichlet(freq_user[i, ] + alpha1, 1).reshape(-1) for i in range(seg_u)])
    oldphi = np.array([np.random.dirichlet(freq_item[j, ] + alpha2, 1).reshape(-1) for j in range(seg_i)])


    ##サンプリング結果の格納とパラメータの表示
    #サンプリング結果の保存
    if rp%keep==0:
        mkeep = rp//keep
        THETA[:, :, mkeep] = oldtheta
        PHI[:, :, mkeep] = oldphi

    #バーンインを超えたらトピックを格納
    if (rp >= burnin) & (rp%keep==0):
        SEG1 += Zi1
        SEG2 += Zi2

    #対数尤度とパラメータの表示
    if rp%disp==0:
        #対数尤度を計算
        LL1 = np.sum(LLi1 * Zi1); LL2 = np.sum(LLi2 * Zi2)
        LL = LL1 + LL2

        #パラメータを表示
        print(rp)
        print(np.round([LL, LL1, LL2], 1))
        print(np.round(np.vstack((r1, pit1)), 3))
        print(np.round(np.vstack((r2, pit2)), 3))

0
[-5395161.6 -2604880.9 -2790280.7]
[[0.13  0.164 0.035 0.248 0.004 0.228 0.001 0.189]
 [0.142 0.122 0.125 0.228 0.045 0.051 0.09  0.198]]
[[0.112 0.129 0.165 0.131 0.01  0.358 0.095]
 [0.141 0.282 0.131 0.089 0.101 0.121 0.134]]
25
[-3567028.3 -1590723.9 -1976304.3]
[[0.122 0.124 0.204 0.085 0.047 0.231 0.135 0.053]
 [0.142 0.122 0.125 0.228 0.045 0.051 0.09  0.198]]
[[0.121 0.13  0.133 0.132 0.377 0.    0.106]
 [0.141 0.282 0.131 0.089 0.101 0.121 0.134]]
50
[-3567036.6 -1590722.  -1976314.7]
[[0.123 0.118 0.193 0.084 0.049 0.226 0.152 0.055]
 [0.142 0.122 0.125 0.228 0.045 0.051 0.09  0.198]]
[[0.12  0.131 0.146 0.153 0.351 0.    0.099]
 [0.141 0.282 0.131 0.089 0.101 0.121 0.134]]
75
[-3567029.  -1590721.1 -1976307.9]
[[0.126 0.124 0.197 0.082 0.049 0.232 0.138 0.052]
 [0.142 0.122 0.125 0.228 0.045 0.051 0.09  0.198]]
[[0.121 0.124 0.151 0.123 0.371 0.001 0.108]
 [0.141 0.282 0.131 0.089 0.101 0.121 0.134]]
100
[-3567043.5 -1590725.4 -1976318.1]
[[0.126 0.119 0.204 0.079 0.05  0.

900
[-3780640.9 -1841419.2 -1939221.6]
[[0.13  0.124 0.206 0.08  0.048 0.227 0.137 0.049]
 [0.142 0.122 0.125 0.228 0.045 0.051 0.09  0.198]]
[[0.124 0.142 0.139 0.125 0.291 0.076 0.102]
 [0.141 0.282 0.131 0.089 0.101 0.121 0.134]]
925
[-3780610.3 -1841414.2 -1939196.2]
[[0.125 0.125 0.203 0.08  0.051 0.224 0.14  0.051]
 [0.142 0.122 0.125 0.228 0.045 0.051 0.09  0.198]]
[[0.13  0.133 0.129 0.125 0.284 0.085 0.115]
 [0.141 0.282 0.131 0.089 0.101 0.121 0.134]]
950
[-3780611.4 -1841413.3 -1939198.1]
[[0.137 0.125 0.194 0.082 0.05  0.228 0.132 0.051]
 [0.142 0.122 0.125 0.228 0.045 0.051 0.09  0.198]]
[[0.13  0.134 0.12  0.129 0.305 0.079 0.105]
 [0.141 0.282 0.131 0.089 0.101 0.121 0.134]]
975
[-3780625.9 -1841417.8 -1939208. ]
[[0.126 0.128 0.198 0.076 0.049 0.231 0.142 0.051]
 [0.142 0.122 0.125 0.228 0.045 0.051 0.09  0.198]]
[[0.123 0.129 0.131 0.129 0.293 0.088 0.106]
 [0.141 0.282 0.131 0.089 0.101 0.121 0.134]]
1000
[-3781627.2 -1841832.9 -1939794.3]
[[0.126 0.121 0.196 0.082 0.

1800
[-3780617.4 -1841415.2 -1939202.2]
[[0.122 0.126 0.202 0.079 0.049 0.231 0.139 0.053]
 [0.142 0.122 0.125 0.228 0.045 0.051 0.09  0.198]]
[[0.118 0.133 0.136 0.125 0.302 0.082 0.106]
 [0.141 0.282 0.131 0.089 0.101 0.121 0.134]]
1825
[-3780632.5 -1841427.5 -1939205. ]
[[0.126 0.125 0.2   0.086 0.049 0.226 0.136 0.052]
 [0.142 0.122 0.125 0.228 0.045 0.051 0.09  0.198]]
[[0.123 0.132 0.136 0.127 0.285 0.095 0.102]
 [0.141 0.282 0.131 0.089 0.101 0.121 0.134]]
1850
[-3784062.6 -1844437.3 -1939625.3]
[[0.128 0.122 0.203 0.082 0.052 0.222 0.141 0.051]
 [0.142 0.122 0.125 0.228 0.045 0.051 0.09  0.198]]
[[0.133 0.12  0.151 0.133 0.288 0.08  0.096]
 [0.141 0.282 0.131 0.089 0.101 0.121 0.134]]
1875
[-3780624.1 -1841412.  -1939212.1]
[[0.128 0.12  0.203 0.086 0.051 0.222 0.141 0.049]
 [0.142 0.122 0.125 0.228 0.045 0.051 0.09  0.198]]
[[0.12  0.137 0.134 0.129 0.292 0.09  0.097]
 [0.141 0.282 0.131 0.089 0.101 0.121 0.134]]
1900
[-3780619.1 -1841420.3 -1939198.7]
[[0.128 0.121 0.201 0.08

2700
[-3780617.9 -1841420.2 -1939197.7]
[[0.127 0.122 0.202 0.08  0.047 0.228 0.139 0.055]
 [0.142 0.122 0.125 0.228 0.045 0.051 0.09  0.198]]
[[0.134 0.133 0.138 0.123 0.29  0.073 0.11 ]
 [0.141 0.282 0.131 0.089 0.101 0.121 0.134]]
2725
[-3780606.2 -1841405.  -1939201.2]
[[0.13  0.118 0.2   0.084 0.049 0.222 0.144 0.052]
 [0.142 0.122 0.125 0.228 0.045 0.051 0.09  0.198]]
[[0.123 0.134 0.146 0.126 0.291 0.074 0.106]
 [0.141 0.282 0.131 0.089 0.101 0.121 0.134]]
2750
[-3780615.1 -1841417.1 -1939198. ]
[[0.128 0.123 0.202 0.083 0.044 0.23  0.14  0.051]
 [0.142 0.122 0.125 0.228 0.045 0.051 0.09  0.198]]
[[0.132 0.129 0.124 0.122 0.303 0.093 0.096]
 [0.141 0.282 0.131 0.089 0.101 0.121 0.134]]
2775
[-3780630.2 -1841420.3 -1939209.8]
[[0.132 0.126 0.191 0.084 0.046 0.227 0.144 0.05 ]
 [0.142 0.122 0.125 0.228 0.045 0.051 0.09  0.198]]
[[0.131 0.136 0.135 0.128 0.291 0.088 0.091]
 [0.141 0.282 0.131 0.089 0.101 0.121 0.134]]
2800
[-3780621.8 -1841420.1 -1939201.8]
[[0.126 0.124 0.198 0.08

3600
[-3780617.1 -1841412.  -1939205.1]
[[0.133 0.122 0.199 0.082 0.045 0.225 0.142 0.051]
 [0.142 0.122 0.125 0.228 0.045 0.051 0.09  0.198]]
[[0.132 0.124 0.14  0.127 0.293 0.083 0.101]
 [0.141 0.282 0.131 0.089 0.101 0.121 0.134]]
3625
[-3780604.2 -1841408.7 -1939195.6]
[[0.123 0.125 0.202 0.082 0.046 0.23  0.142 0.051]
 [0.142 0.122 0.125 0.228 0.045 0.051 0.09  0.198]]
[[0.116 0.13  0.137 0.131 0.291 0.089 0.106]
 [0.141 0.282 0.131 0.089 0.101 0.121 0.134]]
3650
[-3780609.1 -1841410.7 -1939198.4]
[[0.124 0.123 0.197 0.087 0.047 0.223 0.145 0.054]
 [0.142 0.122 0.125 0.228 0.045 0.051 0.09  0.198]]
[[0.125 0.141 0.144 0.119 0.272 0.088 0.111]
 [0.141 0.282 0.131 0.089 0.101 0.121 0.134]]
3675
[-3780730.4 -1841481.4 -1939249.1]
[[0.129 0.119 0.194 0.088 0.048 0.229 0.139 0.054]
 [0.142 0.122 0.125 0.228 0.045 0.051 0.09  0.198]]
[[0.124 0.133 0.142 0.122 0.283 0.093 0.103]
 [0.141 0.282 0.131 0.089 0.101 0.121 0.134]]
3700
[-3780618.7 -1841420.2 -1939198.5]
[[0.123 0.126 0.204 0.07

4500
[-3780613.  -1841410.9 -1939202.1]
[[0.127 0.121 0.195 0.083 0.05  0.233 0.14  0.051]
 [0.142 0.122 0.125 0.228 0.045 0.051 0.09  0.198]]
[[0.124 0.14  0.126 0.131 0.282 0.085 0.111]
 [0.141 0.282 0.131 0.089 0.101 0.121 0.134]]
4525
[-3780608.4 -1841415.6 -1939192.8]
[[0.126 0.123 0.192 0.084 0.049 0.227 0.149 0.05 ]
 [0.142 0.122 0.125 0.228 0.045 0.051 0.09  0.198]]
[[0.133 0.131 0.142 0.133 0.279 0.08  0.102]
 [0.141 0.282 0.131 0.089 0.101 0.121 0.134]]
4550
[-3780609.6 -1841413.8 -1939195.8]
[[0.122 0.125 0.203 0.08  0.05  0.228 0.138 0.053]
 [0.142 0.122 0.125 0.228 0.045 0.051 0.09  0.198]]
[[0.127 0.126 0.132 0.135 0.297 0.075 0.108]
 [0.141 0.282 0.131 0.089 0.101 0.121 0.134]]
4575
[-3780628.2 -1841423.3 -1939204.8]
[[0.128 0.119 0.199 0.082 0.052 0.232 0.137 0.052]
 [0.142 0.122 0.125 0.228 0.045 0.051 0.09  0.198]]
[[0.136 0.133 0.128 0.132 0.285 0.083 0.103]
 [0.141 0.282 0.131 0.089 0.101 0.121 0.134]]
4600
[-3780623.8 -1841423.7 -1939200.2]
[[0.127 0.124 0.196 0.07

In [582]:
np.round(oldtheta, 3)

array([[0.467, 0.462, 0.   , 0.   , 0.002, 0.   , 0.069],
       [0.014, 0.027, 0.858, 0.072, 0.017, 0.   , 0.012],
       [0.161, 0.016, 0.002, 0.   , 0.215, 0.605, 0.002],
       [0.007, 0.032, 0.014, 0.732, 0.   , 0.025, 0.19 ],
       [0.   , 0.756, 0.008, 0.094, 0.   , 0.107, 0.035],
       [0.047, 0.02 , 0.83 , 0.101, 0.   , 0.001, 0.   ],
       [0.043, 0.946, 0.001, 0.01 , 0.   , 0.   , 0.   ],
       [0.001, 0.48 , 0.   , 0.265, 0.2  , 0.055, 0.   ]])

In [583]:
np.round(thetat, 3)

array([[0.   , 0.001, 0.946, 0.01 , 0.   , 0.043, 0.   ],
       [0.   , 0.014, 0.032, 0.732, 0.025, 0.006, 0.192],
       [0.012, 0.863, 0.025, 0.075, 0.   , 0.015, 0.009],
       [0.002, 0.   , 0.465, 0.   , 0.   , 0.464, 0.069],
       [0.   , 0.008, 0.756, 0.094, 0.106, 0.   , 0.035],
       [0.2  , 0.   , 0.479, 0.265, 0.055, 0.001, 0.   ],
       [0.   , 0.828, 0.02 , 0.103, 0.001, 0.047, 0.   ],
       [0.214, 0.002, 0.016, 0.   , 0.606, 0.161, 0.002]])